© 2026 Nokia
          Licensed under the BSD 3-Clause Clear License
          SPDX-License-Identifier: BSD-3-Clause-Clear

In [ ]:
!pip install seaborn

In [ ]:
import time
import pandas as pd
from io import StringIO
import seaborn as sns 
import matplotlib.pyplot as plt

### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
# configure: host name, nic, cores, ram, disk according to your needs. 

slice_name= "wan-inter-site-3-nodes-high-capacity-v2" + fablib.get_bastion_username()

node_conf = [
 {'name': "sender", 'nic': 'NIC_Basic', 'site_name': "ATLA", 'cores': 16, 'ram': 32, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "router", 'nic': 'NIC_ConnectX_6', 'site_name': "ATLA", 'cores': 16, 'ram': 128, 'disk': 60, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "receiver", 'nic': 'NIC_ConnectX_6', 'site_name': "WASH", 'cores': 16, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}
]
net_conf = [
 {"name": "net1", "subnet": "10.0.2.0/24", "nodes": [{"name": "sender",   "addr": "10.0.2.100", 'nic': 'NIC_ConnectX_6'}, {"name": "router", "addr": "10.0.2.101", 'nic': 'NIC_ConnectX_6'}]},
 {"name": "net0", "vlan": 100, "hops":"STAR", "bw": 80, "subnet": "10.0.0.0/24", "nodes": [{"name": "router",   "addr": "10.0.0.100", 'nic': 'NIC_ConnectX_6'}, {"name": "receiver", "addr": "10.0.0.101", 'nic': 'NIC_ConnectX_6'}]}
]

route_conf = [
 {"addr": "10.0.0.0/24", "gw": "10.0.2.101", "nodes": ["sender"]}, 
 {"addr": "10.0.2.0/24", "gw": "10.0.0.100", "nodes": ["receiver"]}
]

exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    if n.get('host'):  # safe check, no KeyError
        slice.add_node(
            name=n['name'], site=n['site_name'],
            cores=n['cores'], ram=n['ram'],
            disk=n['disk'], image=n['image'],
            host=n['host']
        )
    else:
        slice.add_node(
            name=n['name'], site=n['site_name'],
            cores=n['cores'], ram=n['ram'],
            disk=n['disk'], image=n['image']
        )


    slice.get_node(n['name']).add_component(model=n['nic'], name="nic1")

In [ ]:
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")
router_node = slice.get_node("router")

In [ ]:
sender_nic = sender_node.get_component(name="nic1")
receiver_nic = receiver_node.get_component(name="nic1")
router_nic = router_node.get_component(name="nic1")

In [ ]:
for idx, n in enumerate(net_conf):

    if n["name"]=="net1":
        continue
    iface_index = 0 

    ifaces = [slice.get_node("router").get_component(name="nic1").get_interfaces()[iface_index], slice.get_node("receiver").get_component(name="nic1").get_interfaces()[iface_index]]

    for iface in ifaces:
        iface.set_mode('auto')
        iface.set_vlan(str(n["vlan"]))

    ns = slice.add_l2network(name=n["name"], interfaces=ifaces, type='L2PTP')
    #ns.set_l2_route_hops(hops=[n["hops"]])
    ns.set_bandwidth(n["bw"])

In [ ]:
for idx, n in enumerate(net_conf):

    if n["name"]=="net0":
        continue
    iface_index = 0 

    ifaces = [slice.get_node("sender").get_component(name="nic1").get_interfaces()[0], slice.get_node("router").get_component(name="nic1").get_interfaces()[1]]

    ns = slice.add_l2network(name=n["name"], interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

In [ ]:
slice = fablib.get_slice(slice_name)

for node in slice.get_nodes():
    # Pin all vCPUs for VM to same Numa node as the component
    node.pin_cpu(component_name="nic1")
    
    # User can also pass in the range of the vCPUs to be pinned 
    #node.pin_cpu(component_name=nic_name, cpu_range_to_pin="0-3")
    
    # Pin memmory for VM to same Numa node as the components
    #node.numa_tune()
    try:
        node.numa_tune()
        
    except Exception as e:
        print(f"Warning: could not pin memory for {node.get_name()}: {e}")
    
    # Reboot the VM
    node.os_reboot()

In [ ]:
slice.wait_ssh(progress=True)

### Install BBR3 Kernel

In [ ]:
pkg_list = ['linux-headers-6.4.0+_6.4.0-g6e321d1c986a-5_amd64.deb',
            'linux-image-6.4.0+_6.4.0-g6e321d1c986a-5_amd64.deb',
            'linux-libc-dev_6.4.0-g6e321d1c986a-5_amd64.deb']

cmd_BBRv3 ="""sudo grub-set-default "Advanced options for Ubuntu>Ubuntu, with Linux 6.4.0"
sudo grub-mkconfig -o /boot/grub/grub.cfg
sudo sed -i 's/^GRUB_DEFAULT=.*/GRUB_DEFAULT=saved/' /etc/default/grub
sudo update-grub
sudo reboot"""

for pkg in pkg_list:
    slice.get_node(name="sender").execute("wget https://github.com/ashutoshs25/bbrv3-kernel/raw/main/{packet}".format(packet=pkg))
    slice.get_node(name="receiver").execute("wget https://github.com/ashutoshs25/bbrv3-kernel/raw/main/{packet}".format(packet=pkg))
    
slice.get_node(name="sender").execute("sudo dpkg -i  *.deb")
slice.get_node(name="sender").execute(cmd_BBRv3)

slice.get_node(name="receiver").execute("sudo dpkg -i  *.deb")
slice.get_node(name="receiver").execute(cmd_BBRv3)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice=fablib.get_slice(slice_name)
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")
router_node = slice.get_node("router")

slice.wait_ssh()

for i in slice.get_interfaces():
    i.config_vlan_iface()
    i.config()

for n in slice.get_nodes():
    n.config()
    #n.execute('sudo node_tools/host_tune_enable_docker.sh', quiet=True, output_file=f"{n.get_name()}.log")

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo DEBIAN_FRONTEND=noninteractive apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        
        # if_name1 = n['name'] + '-' + net['name'] + '-p1'
        # if_name2 = n['name'] + '-' + net['name'] + '-p2'

        if_name1 = n['name'] + '-' + "nic1" + '-p1'
        if_name2 = n['name'] + '-' + "nic1" + '-p2'
        if_name_list=[if_name1, if_name2]
        #if_name_list=[if_name1]
        
        for if_name in if_name_list:            

            try:
                iface = slice.get_interface(if_name)
            except Exception:   
                print(f"{if_name} not found, skipping")  
                continue
            
            iface.ip_link_up()
            if n['addr']:
                print(n['addr'])
                iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
            else:
                iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# make sure all interfaces are brought up
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
#set up static routes
for rt in route_conf:
   for n in rt['nodes']:
       slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Validate base network

Before we run any experiment, we should check the “base” network - before adding any emulated delay or rate limiting - and make sure that it will not be a limiting factor in the experiment.

In [ ]:
## check base delay
_ = slice.get_node("sender").execute("ping -c 5 10.0.0.101")

In [ ]:
# check base capacity (by sending 10 parallel flows, look at their sum throughput)
import time
_ = slice.get_node("receiver").execute("iperf3 -s -1 -D")
time.sleep(5)
_ = slice.get_node("sender").execute("iperf3 -Z -t 30 -i 10 -C bbr1 -P 10 -c 10.0.0.101")

In [ ]:
# also check Linux kernel version on sender
_ = slice.get_node("sender").execute("uname -a")

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
filtered = [
    entry
    for entry in ifaces
    if entry.get("ip_addr") not in (None, "None")
]

In [ ]:
ifaces_new=filtered

In [ ]:
clean_edges = [
    (u, v, attr)
    for (u, v, attr) in edges
    if (u != 'None' and v != 'None')
]

In [ ]:
edges_new=clean_edges

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges_new)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Tune Hosts

In [ ]:
for node in slice.get_nodes():
    node.upload_directory('/home/fabric/work/Internship-work/node_tools','.')
    node.execute('chmod +x node_tools/host_tune.sh')
    node.execute('sudo sysctl net.ipv4.tcp_available_congestion_control')
    node.execute('sudo ./node_tools/host_tune.sh')

In [ ]:
# For DPDK
router_node.upload_directory('/home/fabric/work/DPDK/node_toolsv2','.')
router_node.execute('chmod +x node_toolsv2/install.sh')
router_node.execute('sudo ./node_toolsv2/install.sh')

In [ ]:
# For DPDK
router_node.execute('chmod +x node_toolsv2/grub.sh')
router_node.execute('chmod +x node_toolsv2/apply_vfio_settings.sh')

stdout, stderr = router_node.execute(f"sudo node_toolsv2/grub.sh {router_node.get_ram()}")
stdout, stderr = router_node.execute("sudo node_toolsv2/apply_vfio_settings.sh")

In [ ]:
router_node.execute("pip install gdown")
router_node.execute("~/.local/bin/gdown https://drive.google.com/uc?id=1Cy0gDm-44sKhaEXyEeuKQSx2di7DO1Xd")

In [ ]:
router_node.execute("sudo mount -o ro,loop /home/ubuntu/MLNX_OFED_LINUX-24.10-3.2.5.0-ubuntu22.04-x86_64.iso /mnt/")
router_node.execute("sudo /mnt/mlnxofedinstall --force") 
router_node.execute("sudo /etc/init.d/openibd restart")

In [ ]:
router_node.execute("sudo apt-get update")
router_node.execute("sudo apt-get install -y libibverbs-dev rdma-core ibverbs-utils pkg-config")

In [ ]:
router_node.execute("sudo reboot")
# wait for all nodes to come back up
slice.wait_ssh(progress=True)

In [ ]:
# necessary for DPDK
cmds = """
sudo apt-get update
sudo apt-get install -y libelf-dev libssl-dev libbsd-dev libarchive-dev libfdt-dev libjansson-dev
"""
router_node.execute(cmds)

### Configure the network capacity and delay

In this section, we configure the bottleneck link to have a 40Mbps capacity and 10ms delay. We will initialize the queue size to 32 BDP (although we will change this later).

Then, we validate the new network setting.

In [ ]:
# check base delay
_ = slice.get_node("sender").execute("ping -c 5 receiver")

In [ ]:
# check base capacity (by sending 10 parallel flows, look at their sum throughput)
import time
_ = slice.get_node("receiver").execute("iperf3 -s -1 -D")
time.sleep(5)
_ = slice.get_node("sender").execute("iperf3 -t 60 -i 10 -P 10 -C cubic -c receiver")

### Experiments

In [ ]:
# sudo apt update
# sudo apt install -y build-essential gcc make libssl-dev libsctp-dev

# # 1) remove any old local install that might shadow the new one
# sudo rm -f /usr/local/bin/iperf3
# sudo rm -f /usr/local/lib/libiperf*
# sudo rm -f /usr/local/include/iperf_api.h
# sudo ldconfig


# cd /tmp
# curl -LO https://github.com/esnet/iperf/releases/download/3.19.1/iperf-3.19.1.tar.gz
# tar -xf iperf-3.19.1.tar.gz
# cd iperf-3.19.1

# ./configure
# make -j"$(nproc)"
# sudo make install
# sudo ldconfig

# hash -r                        
# which iperf3                  
# iperf3 --version

In [ ]:
# sudo ethtool -g enp7s0np0
# sudo ethtool -G enp7s0np0 rx 8192 tx 8192

#  ethtool -S enp7s0np0 | grep rx_out_of_buffer

In [ ]:
def build_drop_check_command(expname: str) -> str:
    """
    Build a self-contained bash command that:
      - auto-discovers interfaces (physical + VLANs on them)
      - checks ip-link, ethtool (real drop/error counters), tc qdisc/class
      - writes and tees output to /home/ubuntu/drop_check_<expname>.log
      - exits 0 if clean, 1 if any drops/errors found
    """
    # Pass expname via a shell var to avoid Python string formatting conflicts
    prefix = f'EXP="{expname}"; '
    body = r'''
LOGFILE="/home/ubuntu/drop_check_${EXP}.log"
exec > >(tee -a "$LOGFILE") 2>&1

echo "=== Drop Check Started at $(date) ==="

# --- Auto-discover interfaces (physical + VLANs on them) ---
mapfile -t PHYS_IFACES < <(for d in /sys/class/net/*; do
  dev=$(basename "$d")
  [[ "$dev" == "lo" ]] && continue
  [[ -e "$d/device" ]] || continue
  echo "$dev"
done)

mapfile -t VLAN_IFACES < <(ip -d -o link show type vlan 2>/dev/null | awk -F': ' '{print $2}' | while read -r pair; do
  ifname="${pair%%@*}"; lower="${pair##*@}"
  for p in "${PHYS_IFACES[@]}"; do
    if [[ "$lower" == "$p" ]]; then echo "$ifname"; break; fi
  done
done)

declare -A seen
IFACES=()
for x in "${PHYS_IFACES[@]}" "${VLAN_IFACES[@]}"; do
  [[ -n "$x" && -d "/sys/class/net/$x" && -z "${seen[$x]}" ]] && IFACES+=("$x") && seen[$x]=1
done

echo "Discovered interfaces: ${IFACES[*]}"
ok=true

# --- Checkers ---
check_ip_link() {
  ip -s link show dev "$1" 2>/dev/null | awk '
    /RX:/ { getline; split($0,a); rx_err=a[3]+0; rx_drop=a[4]+0 }
    /TX:/ { getline; split($0,b); tx_err=b[3]+0; tx_drop=b[4]+0 }
    END {
      if (rx_err || rx_drop || tx_err || tx_drop)
        printf "ip_link: RXerr=%d RXdrop=%d TXerr=%d TXdrop=%d\n", rx_err, rx_drop, tx_err, tx_drop
    }'
}

check_ethtool() {
  local IFACE="$1"
  local BASE="${IFACE%%.*}"
  (sudo ethtool -S "$IFACE" 2>/dev/null || sudo ethtool -S "$BASE" 2>/dev/null) | \
  awk -F':' '
    {
      k=$1; v=$2; gsub(/^[ \t]+|[ \t]+$/, "", v); lk=tolower(k)
      # Only counters that indicate *real* loss/errors; print only if >0
      if (lk ~ /(rx_out_of_buffer|rx_dropped|tx_dropped|rx_errors|tx_errors|no_buffer|overflow|rx_missed_errors|discard|no_desc|fifo_errors|crc_errors|length_errors)/) {
        if ((v+0) > 0) printf "ethtool: %s %s\n", k, v+0
      }
    }'
}

check_tc_qdisc_class() {
  local IFACE="$1"
  sudo tc -s qdisc show dev "$IFACE" 2>/dev/null | \
    awk '/Sent/ { if (match($0,/dropped +([0-9]+)/,m) && m[1]>0) print "qdisc: " $0 }'
  sudo tc -s class show dev "$IFACE" 2>/dev/null | \
    awk '/Sent/ { if (match($0,/dropped +([0-9]+)/,m) && m[1]>0) print "class: " $0 }'
}

# --- Run checks ---
for IFACE in "${IFACES[@]}"; do
  echo "=== Checking $IFACE ==="
  DROPS=$(
    check_ip_link "$IFACE"
    check_ethtool "$IFACE"
    check_tc_qdisc_class "$IFACE"
  )
  if [[ -z "$DROPS" ]]; then
    echo "✅ No drops on $IFACE"
  else
    ok=false
    echo "❌ Drops/errors detected on $IFACE:"
    echo "$DROPS"
  fi
  echo
done

if $ok; then
  echo "✅✅ All interfaces clean — No drops anywhere."
  rc=0
else
  echo "⚠️ Some interfaces show drops/errors above."
  rc=1
fi

echo "=== Drop Check Finished at $(date) ==="
echo "Results saved to $LOGFILE"
exit $rc
'''
    return prefix + body


def run_drop_check(node, expname: str):
    """Execute the drop-check on a given node with a chosen experiment name."""
    cmd = build_drop_check_command(expname)
    return node.execute(cmd)

In [ ]:
def build_drop_check_command(expname: str) -> str:
    """
    Build a self-contained bash command that:
      - auto-discovers interfaces (physical + VLANs on them)
      - checks ip-link, ethtool (real drop/error counters), tc qdisc/class
      - checks per-CPU softnet backlog drops and (if available) nstat TCP/IP counters
      - writes and tees output to /home/ubuntu/drop_check_<expname>.log
      - exits 0 if clean, 1 if any drops/errors found
    """
    prefix = f'EXP="{expname}"; '
    body = r'''
LOGFILE="/home/ubuntu/drop_check_${EXP}.log"
exec > >(tee -a "$LOGFILE") 2>&1

echo "=== Drop Check Started at $(date) ==="

# --- Auto-discover interfaces (physical + VLANs on them) ---
mapfile -t PHYS_IFACES < <(for d in /sys/class/net/*; do
  dev=$(basename "$d")
  [[ "$dev" == "lo" ]] && continue
  [[ -e "$d/device" ]] || continue   # only devices backed by hardware
  echo "$dev"
done)

mapfile -t VLAN_IFACES < <(ip -d -o link show type vlan 2>/dev/null | awk -F': ' '{print $2}' | while read -r pair; do
  ifname="${pair%%@*}"; lower="${pair##*@}"
  for p in "${PHYS_IFACES[@]}"; do
    if [[ "$lower" == "$p" ]]; then echo "$ifname"; break; fi
  done
done)

declare -A seen
IFACES=()
for x in "${PHYS_IFACES[@]}" "${VLAN_IFACES[@]}"; do
  [[ -n "$x" && -d "/sys/class/net/$x" && -z "${seen[$x]}" ]] && IFACES+=("$x") && seen[$x]=1
done

echo "Discovered interfaces: ${IFACES[*]}"
ok=true

# --- Helpers ---

check_ip_link() {
  ip -s link show dev "$1" 2>/dev/null | awk '
    /RX:/ { getline; split($0,a); rx_err=a[3]+0; rx_drop=a[4]+0 }
    /TX:/ { getline; split($0,b); tx_err=b[3]+0; tx_drop=b[4]+0 }
    END {
      if (rx_err || rx_drop || tx_err || tx_drop)
        printf "ip_link: RXerr=%d RXdrop=%d TXerr=%d TXdrop=%d\n", rx_err, rx_drop, tx_err, tx_drop
    }'
}

check_ethtool() {
  local IFACE="$1"
  local BASE="${IFACE%%.*}"
  (sudo ethtool -S "$IFACE" 2>/dev/null || sudo ethtool -S "$BASE" 2>/dev/null) | \
  awk -F':' '
    {
      k=$1; v=$2; gsub(/^[ \t]+|[ \t]+$/, "", v); lk=tolower(k)
      # Print only if >0; focus on true loss/error indicators.
      if (lk ~ /(rx_out_of_buffer|rx_dropped|tx_dropped|rx_errors|tx_errors|no_buffer|overflow|rx_missed_errors|discard|no_desc|fifo_errors|crc_errors|length_errors|rx_discards_phy|rx_prio[0-9]+_discards)/) {
        if ((v+0) > 0) printf "ethtool: %s %s\n", k, v+0
      }
    }'
}

check_tc_qdisc_class() {
  local IFACE="$1"
  sudo tc -s qdisc show dev "$IFACE" 2>/dev/null | \
    awk '/Sent/ { if (match($0,/dropped +([0-9]+)/,m) && m[1]>0) print "qdisc: " $0 }'
  sudo tc -s class show dev "$IFACE" 2>/dev/null | \
    awk '/Sent/ { if (match($0,/dropped +([0-9]+)/,m) && m[1]>0) print "class: " $0 }'
}

check_softnet_stat() {
  # Snapshot (no delta): column 2 per line (hex) = softnet backlog drops on that CPU.
  i=0
  while read -r line; do
    i=$((i+1)); set -- $line
    drops=$((16#$2))
    if (( drops > 0 )); then
      echo "softnet: CPU$((i-1)) backlog_drops(total) = $drops"
    fi
  done < /proc/net/softnet_stat
}

check_nstat() {
  # Optional: shows TCP/IP stack counters if iproute2's nstat is installed.
  if ! command -v nstat >/dev/null 2>&1; then
    return 0
  fi
  nstat -az 2>/dev/null | awk '
    $1 ~ /^(IpInDiscards|IpInErrors|TcpInErrs|TcpRetransSegs)$/ && $2>0 {
      printf "nstat: %s = %s\n", $1, $2
    }'
}

# --- Run per-interface checks ---
for IFACE in "${IFACES[@]}"; do
  echo "=== Checking $IFACE ==="
  DROPS=$(
    check_ip_link "$IFACE"
    check_ethtool "$IFACE"
    check_tc_qdisc_class "$IFACE"
  )
  if [[ -z "$DROPS" ]]; then
    echo "✅ No drops on $IFACE"
  else
    ok=false
    echo "❌ Drops/errors detected on $IFACE:"
    echo "$DROPS"
  fi
  echo
done

# --- System-wide checks ---
echo "=== Checking softnet backlog drops (per-CPU) ==="
SN=$(check_softnet_stat)
if [[ -z "$SN" ]]; then
  echo "✅ No softnet backlog drops"
else
  ok=false
  echo "❌ Softnet drops detected:"
  echo "$SN"
fi
echo

echo "=== Checking nstat (TCP/IP stack counters) ==="
NS=$(check_nstat)
if [[ -z "$NS" ]]; then
  echo "✅ nstat clean or not installed"
else
  ok=false
  echo "❌ Stack-level counters nonzero:"
  echo "$NS"
fi
echo

# --- Result ---
if $ok; then
  echo "✅✅ All interfaces clean — No drops anywhere."
  rc=0
else
  echo "⚠️ Some drops/errors detected above."
  rc=1
fi

echo "=== Drop Check Finished at $(date) ==="
echo "Results saved to $LOGFILE"
exit $rc
'''
    return prefix + body


def run_drop_check(node, expname: str):
    """Execute the drop-check on a given node with a chosen experiment name."""
    cmd = build_drop_check_command(expname)
    return node.execute(cmd)


In [ ]:
import re

METRICS = [
    "rx_out_of_buffer",
    "rx_discards_phy",
    "rx_prio_discards",
    "ip_rxdrop",
    "ip_txdrop",
    "tc_qdisc_drop",
    "tc_class_drop",
    "retrans",
]


def parse_drop_log(log_text: str) -> dict:
    """
    Parse a single node's drop_check log into host-level counters.

    - Tracks per-interface stats under the current "=== Checking IFACE ===".
    - Groups interfaces by base name (e.g., enp7s0np0 and enp7s0np0.100 → base 'enp7s0np0').
    - For each base, takes the *max* across its interfaces (to avoid double-counting VLAN mirrors).
    - Sums across base interfaces to get host-level totals.
    - 'retrans' (TcpRetransSegs) is node-wide, not per-interface.
    """
    iface_counters = {}  # iface -> metric -> value
    host_retrans = 0
    current_iface = None

    def ensure_iface(iface):
        if iface not in iface_counters:
            iface_counters[iface] = {
                "rx_out_of_buffer": 0,
                "rx_discards_phy": 0,
                "rx_prio_discards": 0,
                "ip_rxdrop": 0,
                "ip_txdrop": 0,
                "tc_qdisc_drop": 0,
                "tc_class_drop": 0,
            }

    for raw in log_text.splitlines():
        line = raw.strip()

        # Detect interface section
        m = re.match(r"=== Checking (\S+) ===", line)
        if m:
            current_iface = m.group(1)
            ensure_iface(current_iface)
            continue

        # nstat (host-wide)
        m = re.match(r"nstat:\s+TcpRetransSegs\s+=\s+(\d+)", line)
        if m:
            host_retrans = int(m.group(1))
            continue

        if not current_iface:
            continue  # ignore lines outside an interface section

        # ip -s link RX/TX drops
        m = re.match(r"ip_link: RXerr=\d+ RXdrop=(\d+) TXerr=\d+ TXdrop=(\d+)", line)
        if m:
            rx_drop = int(m.group(1))
            tx_drop = int(m.group(2))
            ensure_iface(current_iface)
            iface_counters[current_iface]["ip_rxdrop"] += rx_drop
            iface_counters[current_iface]["ip_txdrop"] += tx_drop
            continue

        # ethtool stats
        m = re.match(r"ethtool:\s+(\S+)\s+(\d+)", line)
        if m:
            key = m.group(1)
            val = int(m.group(2))
            ensure_iface(current_iface)
            if key == "rx_out_of_buffer":
                # absolute counter
                iface_counters[current_iface]["rx_out_of_buffer"] = val
            elif key == "rx_discards_phy":
                iface_counters[current_iface]["rx_discards_phy"] = val
            elif re.match(r"rx_prio\d+_discards", key):
                # could be multiple priorities; sum them within the iface
                iface_counters[current_iface]["rx_prio_discards"] += val
            continue

        # tc qdisc drops
        m = re.search(r"qdisc: .*dropped (\d+)", line)
        if m:
            ensure_iface(current_iface)
            iface_counters[current_iface]["tc_qdisc_drop"] += int(m.group(1))
            continue

        # tc class drops
        m = re.search(r"class: .*dropped (\d+)", line)
        if m:
            ensure_iface(current_iface)
            iface_counters[current_iface]["tc_class_drop"] += int(m.group(1))
            continue

    # --- Group by base interface (e.g., enp7s0np0 vs enp7s0np0.100) ---
    grouped = {}  # base_iface -> metric -> value (max across variants)
    for iface, stats in iface_counters.items():
        base = iface.split(".")[0]
        if base not in grouped:
            grouped[base] = {k: 0 for k in stats.keys()}
        for k, v in stats.items():
            if v > grouped[base][k]:
                grouped[base][k] = v

    # --- Sum across base interfaces to get host-level totals ---
    total = {k: 0 for k in METRICS}
    for base, stats in grouped.items():
        for k in stats:
            total[k] += stats[k]

    total["retrans"] = host_retrans
    return total


def print_counter_diff(label: str, before: dict, after: dict):
    print(f"\n=== DIFF for {label} ===")
    for k in METRICS:
        b = before.get(k, 0)
        a = after.get(k, 0)
        diff = a - b
        print(f"{k:18s}: before={b:8d}  after={a:8d}  diff={diff:+d}")
    print("====================================\n")


In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
router_ingress_iface = router_node.get_interface(network_name = "net1")
router_ingress_name = router_ingress_iface.get_device_name()
router_egress_iface  = router_node.get_interface(network_name = "net0")
router_egress_name = router_egress_iface.get_device_name()

In [ ]:
sender_egress_iface = sender_node.get_interface(network_name = "net1")
sender_egress_name = sender_egress_iface.get_device_name()

receiver_ingress_iface  = receiver_node.get_interface(network_name = "net0")
receiver_ingress_name = receiver_ingress_iface.get_device_name()

In [ ]:
sender_node.execute("uname -r")
receiver_node.execute("uname -r")

In [ ]:
sender_node.execute("iperf3 --version")
receiver_node.execute("iperf3 --version")

In [ ]:
# generate full factorial experiment
import itertools

exp_factors = {
    'n_bdp': [8],  # n x bandwidth delay product (BDP)
    'btl_capacity': [80], #in Gbps 
    'burst':[13000],
    'base_rtt': [13], # in ms 
    #'aqm': ['FIFO', 'single-queue-FQ', 'pie_drop', 'Codel_drop', 'Codel', 'FQ', 'FQ_Codel', 'FQ_Codel_L4S', 'DualPI2'],
    'aqm': ['FIFO'],
    'ecn_threshold': [52], # in ms
    'ecn': [1],  # 0: noecn, 1: ecn, 3: accecn #'rx_L4S_ecn': [0, 1, 3]
    'cc': ["bbr1"], # prague, bbr2, cubic, bbr
    'flow_number': [10], # flow per file
    'duration': [120],
    'file_size':[5], # in GB
    'trial': [1,2,3,4,5,6,7,8,9,10] # it means total 10 trials here.
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))

exp_lists = []

seen_combinations = set()

# Removing ECN factor from FIFO bottleneck because it does not support ECN. This could be done also for pie_drop and codel_drop but we have not done.
# Removing the cases where ECN Threshold is less than or equal to the buffer size in time, these cases are not meaningful in practice.

for factor_l in factor_lists:
    temp_dict = dict(zip(factor_names, factor_l))
    if temp_dict['n_bdp'] * temp_dict['base_rtt'] >= temp_dict['ecn_threshold']:
        if temp_dict['aqm'] == 'FIFO':
            del temp_dict['ecn_threshold']
        # Convert dict to a frozenset for set operations
        fs = frozenset(temp_dict.items())
    
        if fs not in seen_combinations:
            seen_combinations.add(fs)
            exp_lists.append(temp_dict)

data_dir = slice_name + '-same-site'

print("Number of experiments:",len(exp_lists))

In [ ]:
# sender_node.execute('sudo sysctl -w net.core.wmem_default=$((16*1024*1024))')
# sender_node.execute('sudo sysctl -w net.core.rmem_default=$((16*1024*1024))')


# sender_node.execute('sudo sysctl -w net.core.wmem_max=$((512*1024*1024))')
# sender_node.execute('sudo sysctl -w net.core.rmem_max=$((512*1024*1024))')

# sender_node.execute('sudo sysctl -w net.ipv4.tcp_wmem="$((6*1024*1024)) $((16*1024*1024)) $((256*1024*1024))"')
# sender_node.execute('sudo sysctl -w net.ipv4.tcp_rmem="$((6*1024*1024)) $((16*1024*1024)) $((256*1024*1024))"')

In [ ]:
# --- Socket / TCP memory ceilings (match table's "2147483647") ---
sender_node.execute('sudo sysctl -w net.core.wmem_max=2147483647')
sender_node.execute('sudo sysctl -w net.core.rmem_max=2147483647')

# tcp_mem is in PAGES (low / pressure / high) – set very high as in the table
sender_node.execute('sudo sysctl -w net.ipv4.tcp_mem="2147483647 2147483647 2147483647"')

# TCP autotune ranges (min, default, max in BYTES) – max to 2 GB per table
sender_node.execute('sudo sysctl -w net.ipv4.tcp_wmem="4096 87380 2147483647"')
sender_node.execute('sudo sysctl -w net.ipv4.tcp_rmem="4096 87380 2147483647"')

sender_node.execute('sudo sysctl -w net.core.wmem_default=$((16*1024*1024))')
sender_node.execute('sudo sysctl -w net.core.rmem_default=$((16*1024*1024))')

# --- Other kernel knobs from the table ---
sender_node.execute('sudo sysctl -w net.core.netdev_max_backlog=250000')
sender_node.execute('sudo sysctl -w net.ipv4.tcp_mtu_probing=1')
sender_node.execute('sudo sysctl -w net.ipv4.tcp_no_metrics_save=1')
sender_node.execute('sudo sysctl -w net.ipv4.tcp_timestamps=1')
sender_node.execute('sudo sysctl -w net.core.default_qdisc=fq')

In [ ]:
# receiver_node.execute('sudo sysctl -w net.core.wmem_default=$((16*1024*1024))')
# receiver_node.execute('sudo sysctl -w net.core.rmem_default=$((16*1024*1024))')

# receiver_node.execute('sudo sysctl -w net.core.wmem_max=$((512*1024*1024))')
# receiver_node.execute('sudo sysctl -w net.core.rmem_max=$((512*1024*1024))')

# receiver_node.execute('sudo sysctl -w net.ipv4.tcp_wmem="$((6*1024*1024)) $((16*1024*1024)) $((256*1024*1024))"')
# receiver_node.execute('sudo sysctl -w net.ipv4.tcp_rmem="$((6*1024*1024)) $((16*1024*1024)) $((256*1024*1024))"')

In [ ]:
# --- Socket / TCP memory ceilings (match table's "2147483647") ---
receiver_node.execute('sudo sysctl -w net.core.wmem_max=2147483647')
receiver_node.execute('sudo sysctl -w net.core.rmem_max=2147483647')

# tcp_mem is in PAGES (low / pressure / high) – set very high as in the table
receiver_node.execute('sudo sysctl -w net.ipv4.tcp_mem="2147483647 2147483647 2147483647"')

# TCP autotune ranges (min, default, max in BYTES) – max to 2 GB per table
receiver_node.execute('sudo sysctl -w net.ipv4.tcp_wmem="4096 87380 2147483647"')
receiver_node.execute('sudo sysctl -w net.ipv4.tcp_rmem="4096 87380 2147483647"')

receiver_node.execute('sudo sysctl -w net.core.wmem_default=$((16*1024*1024))')
receiver_node.execute('sudo sysctl -w net.core.rmem_default=$((16*1024*1024))')

# --- Other kernel knobs from the table ---
receiver_node.execute('sudo sysctl -w net.core.netdev_max_backlog=250000')
receiver_node.execute('sudo sysctl -w net.ipv4.tcp_mtu_probing=1')
receiver_node.execute('sudo sysctl -w net.ipv4.tcp_no_metrics_save=1')
receiver_node.execute('sudo sysctl -w net.ipv4.tcp_timestamps=1')
receiver_node.execute('sudo sysctl -w net.core.default_qdisc=fq')

In [ ]:
# check the buffer sizes
sender_node.execute("sysctl net.core.rmem_max net.core.wmem_max net.ipv4.tcp_rmem net.ipv4.tcp_wmem")
receiver_node.execute("sysctl net.core.rmem_max net.core.wmem_max net.ipv4.tcp_rmem net.ipv4.tcp_wmem")

In [ ]:
import time


# make sure that you are using correct kernel. 

# make sure BBR is available # this is BBRv3 in BBRv3 kernel, otherwise it is BBRv1. 
# In BBRv3 kernel, BBRv1 is bbr1 not bbr.


#sender_node.execute("sudo modprobe tcp_bbr") 
#receiver_node.execute("sudo modprobe tcp_bbr")

old_flow_number=0

for exp in exp_lists:

    if exp['aqm']=="FIFO":
        exp['ecn_threshold']=0
    
        
    name = str(exp['cc'])+"_" +str(exp['aqm']) +"_" +str(exp['ecn']) +"_" +str(exp['ecn_threshold']) +"_" +str(exp['n_bdp'])+"_"+str(exp['btl_capacity'])+"_" +str(exp['burst'])+"_"+str(exp['base_rtt'])+"_"+ str(exp['flow_number']) +"_" +str(exp['duration'])+"_"+str(exp['file_size']) +"_"+str(exp['ssthresh_bdp_level'])+"_"+str(exp['reset_timer'])+"_"+str(exp['trial'])
    name_tx= name+ ".txt"
    
    
    stdout_tx_json, stderr_tx_json = receiver_node.execute("ls " + name_tx, quiet=True) 
    

    if len(stdout_tx_json):
        print("Already have " + name_tx + ", skipping")

    elif len(stderr_tx_json):

        print("Running:",exp)

        sender_node.execute("sudo modprobe tcp_" + exp['cc'])

        if exp['cc'] =="bbr3":
            
            exp['cc'] = "bbr"
            

        receiver_address="10.0.0.101"
    
            
        # output = receiver_node.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer")[0].strip()
        # print(f"receiver enp7s0np0: {output}")

        # output = sender_node.execute("ethtool -S enp7s0 | grep rx_out_of_buffer")[0].strip()
        # print(f"sender enp7s0: {output}")

        # output = router_node.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer")[0].strip()
        # print(f"router enp7s0np0: {output}")

        # output = router_node.execute("ethtool -S enp8s0np1 | grep rx_out_of_buffer")[0].strip()
        # print(f"router enp8s0np1: {output}")

        # run_drop_check(router_node, name+"_before")
        # run_drop_check(sender_node, name+"_before")
        # run_drop_check(receiver_node, name+"_before")

        

        receiver_node.execute("sudo modprobe tcp_" + exp['cc'])
        receiver_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc'])

        receiver_node.execute("sudo sysctl -w net.ipv4.tcp_ecn="+str(exp['ecn'])) 
        
        # fixed values
        btl_limit    = int(1000*exp['n_bdp']*exp['btl_capacity']*1000*exp['base_rtt']/8) # limit of the bottleneck, n_bdp x BDP in bytes 
        packet_number=int(btl_limit/9000)+1

        # set router buffer limit 
        bdp_kbyte = exp['base_rtt']*exp['btl_capacity']*1000/8
        btl_limit_ss=int(1000*1000*exp['btl_capacity']*(exp['base_rtt'])/8)

        btl_limit_ss=(btl_limit_ss/9000)

        ssthreshinit=btl_limit_ss # BDP initialization

        
        sender_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc'])
        sender_node.execute("sudo sysctl -w net.ipv4.tcp_ecn="+str(exp['ecn'])) 

        cmds_prefix = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}Gbit ceil {capacity}Gbit burst {burst}kb cburst {burst}kb quantum 9000
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], burst=exp['burst'])

        cmds_specific_initial = "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: ".format(iface=router_egress_name)
  
        cmds_specific = {
        'FIFO': "bfifo limit {buffer}".format(buffer=btl_limit),
        #'FQ': "fq limit 41667 flow_limit 41667 ce_threshold {threshold}ms".format(packet_limit=min(1000000,packet_number), threshold=exp.get('ecn_threshold', 0)),
        'FQ': "fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'single-queue-FQ': "fq limit {packet_limit} flow_limit {packet_limit} orphan_mask 0 quantum 9000 initial_quantum 9000 ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'Codel': "codel limit {packet_limit} target {target}ms interval 100ms ecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'FQ_new': "fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms".format(packet_limit=min(1000000,packet_number), threshold=exp.get('ecn_threshold', 0)),
        'FQ_no_ecn': "fq limit {packet_limit} flow_limit {packet_limit}".format(packet_limit=min(10000,packet_number)),
        'FQ_Codel': "fq_codel limit {packet_limit} target {target}ms interval 100ms ecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        #'DualPI2': "dualpi2 limit {packet_limit} alpha 0 beta 0 target 1000ms step_thresh {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0))
        'DualPI2': "dualpi2 limit {packet_limit} target {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0))
        }



        cmds_aqm = {key: cmds_specific_initial + cmd for key, cmd in cmds_specific.items()}

        router_node.execute(f"sudo tc qdisc del dev {router_egress_name} root")

        
        router_node.execute(cmds_prefix)
        router_node.execute(cmds_aqm[ exp['aqm'] ])


        # clean up
        receiver_node.execute("sudo killall iperf3")
        sender_node.execute("sudo killall iperf3")
        
        time.sleep(5) 
        
        # start an iperf3 receiver
        receiver_node.execute_thread(f"iperf3 -s -1 -i 0.1 -p 5201 -fm --logfile " + name + ".txt")
    
        time.sleep(10) 
    
        ss_tx_script="rm -f {flow}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipmn dst 10.0.0.101:5201 | ts '%.s' | tee -a {flow}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"      

        queue_script="sleep 4; rm {flow}-queue.txt; start_time=$(date +%s); while true; do tc -p -s -d qdisc show dev {iface} | tr -d \'\n\' | ts '%.s' | tee -a {flow}-queue.txt; echo \"\" | tee -a {flow}-queue.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.01; done;"
        


        # set time for transmission
        sender_node.execute_thread(f"sleep 1; iperf3 -Z -c {receiver_address} -p 5201 -fm -t " + str(exp['duration']) + " -C " + exp["cc"] + " -P "+str(exp['flow_number']) +" -J > "+ name+".json")

        # set data size for transmission
        #sender_node.execute("sleep 1; iperf3 -c receiver -p 5201 -n " + str(exp['file_size']) + "G"+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) +" -J > "+ name+".json")

        time.sleep(exp['duration'] + 10)


        receiver_node.execute(f"tail -n 1 {name}.txt")


        # run_drop_check(router_node, name)
        # run_drop_check(sender_node, name)
        # run_drop_check(receiver_node, name)


        #         # --- ROUTER before/after ---
        # before_router_log = router_node.execute(f"cat /home/ubuntu/drop_check_{name}_before.log")[0]
        # after_router_log  = router_node.execute(f"cat /home/ubuntu/drop_check_{name}.log")[0]
        
        # before_router = parse_drop_log(before_router_log)
        # after_router  = parse_drop_log(after_router_log)
        
        # # --- SENDER before/after ---
        # before_sender_log = sender_node.execute(f"cat /home/ubuntu/drop_check_{name}_before.log")[0]
        # after_sender_log  = sender_node.execute(f"cat /home/ubuntu/drop_check_{name}.log")[0]
        
        # before_sender = parse_drop_log(before_sender_log)
        # after_sender  = parse_drop_log(after_sender_log)
        
        # # --- RECEIVER before/after ---
        # before_receiver_log = receiver_node.execute(f"cat /home/ubuntu/drop_check_{name}_before.log")[0]
        # after_receiver_log  = receiver_node.execute(f"cat /home/ubuntu/drop_check_{name}.log")[0]
        
        # before_receiver = parse_drop_log(before_receiver_log)
        # after_receiver  = parse_drop_log(after_receiver_log)
        
        # # --- Print comparisons ---
        # print_counter_diff("ROUTER",   before_router,   after_router)
        # print_counter_diff("SENDER",   before_sender,   after_sender)
        # print_counter_diff("RECEIVER", before_receiver, after_receiver)

    
print("done")

# output = receiver_node.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer")[0].strip()
# print(f"receiver enp7s0np0: {output}")

# output = sender_node.execute("ethtool -S enp7s0 | grep rx_out_of_buffer")[0].strip()
# print(f"sender enp7s0: {output}")

# output = router_node.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer")[0].strip()
# print(f"router enp7s0np0: {output}")

# output = router_node.execute("ethtool -S enp8s0np1 | grep rx_out_of_buffer")[0].strip()
# print(f"router enp8s0np1: {output}")

In [ ]:
import time


# make sure that you are using correct kernel. 


# make sure BBR is available # this is BBRv3 in BBRv3 kernel, otherwise it is BBRv1. 
# In BBRv3 kernel, BBRv1 is bbr1 not bbr.


#sender_node.execute("sudo modprobe tcp_bbr") 
#receiver_node.execute("sudo modprobe tcp_bbr")

old_flow_number=0

for exp in exp_lists:

    if exp['aqm']=="FIFO":
        exp['ecn_threshold']=0


    if exp['cc']=="prague":
        exp['ecn']=3
        
    name = str(exp['cc'])+"_" +str(exp['aqm']) +"_" +str(exp['ecn']) +"_" +str(exp['ecn_threshold']) +"_" +str(exp['n_bdp'])+"_"+str(exp['btl_capacity'])+"_"+str(exp['base_rtt'])+"_"+ str(exp['flow_number']) +"_" +str(exp['duration'])+"_"+str(exp['file_size']) +"_"+str(exp['ssthresh_bdp_level'])+"_"+str(exp['reset_timer'])+"_"+str(exp['trial'])
    name_tx= name+ ".txt"
    
    
    stdout_tx_json, stderr_tx_json = receiver_node.execute("ls " + name_tx, quiet=True) 
    

    if len(stdout_tx_json):
        print("Already have " + name_tx + ", skipping")

    elif len(stderr_tx_json):

        print("Running:",exp)

        receiver_address="10.0.0.101"


        # output = receiver_node.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer")[0].strip()
        # print(f"receiver enp7s0np0: {output}")

        # output = sender_node.execute("ethtool -S enp7s0 | grep rx_out_of_buffer")[0].strip()
        # print(f"sender enp8s0np1: {output}")

        # run_drop_check(router_node, name+"_before")
        # run_drop_check(sender_node, name+"_before")
        # run_drop_check(receiver_node, name+"_before")

        if exp['cc'] =="bbr3":
            
            exp['cc'] = "bbr"

        sender_node.execute("sudo modprobe tcp_" + exp['cc'])

        receiver_node.execute("sudo modprobe tcp_" + exp['cc'])
        receiver_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc'])

        receiver_node.execute("sudo sysctl -w net.ipv4.tcp_ecn="+str(exp['ecn'])) 
        
        # fixed values
        btl_limit    = int(1000*exp['n_bdp']*exp['btl_capacity']*1000*exp['base_rtt']/8) # limit of the bottleneck, n_bdp x BDP in bytes 
        packet_number=int(btl_limit/1500)+1

        # set router buffer limit 
        bdp_kbyte = exp['base_rtt']*exp['btl_capacity']*1000/8

        btl_limit_ss=int(1000*1000*exp['btl_capacity']*(exp['base_rtt'])/8)

        
        btl_limit_ss=(btl_limit_ss/1500)

        
        ssthreshinit=btl_limit_ss # BDP initialization
    
        
        sender_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc'])
        sender_node.execute("sudo sysctl -w net.ipv4.tcp_ecn="+str(exp['ecn'])) 

        
        cmds_specific = {
        'FIFO': "bfifo limit {buffer}".format(buffer=btl_limit),
        #'FQ': "fq limit 41667 flow_limit 41667 ce_threshold {threshold}ms".format(packet_limit=min(1000000,packet_number), threshold=exp.get('ecn_threshold', 0)),
        'FQ': "fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'single-queue-FQ': "fq limit {packet_limit} flow_limit {packet_limit} orphan_mask 0 ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'FQ_new': "fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms".format(packet_limit=min(1000000,packet_number), threshold=exp.get('ecn_threshold', 0)),
        'FQ_no_ecn': "fq limit {packet_limit} flow_limit {packet_limit}".format(packet_limit=min(10000,packet_number)),
        'FQ_Codel': "fq_codel limit {packet_limit} target {target}ms interval 100ms ecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'DualPI2': "dualpi2 limit {packet_limit} alpha 0 beta 0 target 1000ms step_thresh {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0))
        }

        # clean up
        receiver_node.execute("sudo killall iperf3")
        sender_node.execute("sudo killall iperf3")
        
        time.sleep(5) 


        router_node.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm01_1flow.cfg 1 > output-{name}.log 2>&1") 

        time.sleep(1) 

        router_node.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.10/dpdk-stable-22.11.10/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 1 > output-reverse-{name}.log 2>&1")

        time.sleep(1)
        
        # start an iperf3 receiver
        receiver_node.execute_thread(f"iperf3 -s -1 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")

    
        time.sleep(10) 
    
      

        sender_node.execute_thread("sleep 1; iperf3 -Z -c 10.0.0.101 -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) +" -J > "+ name+".json")
        

        time.sleep(exp['duration'] + 10)

        router_node.execute("sudo pkill -f tm10")
        router_node.execute("pkill -f 'mpstat -P ALL 1'")

        router_node.execute("sudo rm -f /mnt/huge/rte*map_*")

        router_node.execute("sudo rm -f /mnt/huge1G/rte*map_*")

        router_node.execute("sudo rm -f /dev/hugepages/rte*map_*")

        receiver_node.execute("sudo killall iperf3")
        sender_node.execute("sudo killall iperf3")


        receiver_node.execute(f"tail -n 1 {name}.txt")

        # run_drop_check(router_node, name)
        # run_drop_check(sender_node, name)
        # run_drop_check(receiver_node, name)


        #         # --- ROUTER before/after ---
        # before_router_log = router_node.execute(f"cat /home/ubuntu/drop_check_{name}_before.log")[0]
        # after_router_log  = router_node.execute(f"cat /home/ubuntu/drop_check_{name}.log")[0]
        
        # before_router = parse_drop_log(before_router_log)
        # after_router  = parse_drop_log(after_router_log)
        
        # # --- SENDER before/after ---
        # before_sender_log = sender_node.execute(f"cat /home/ubuntu/drop_check_{name}_before.log")[0]
        # after_sender_log  = sender_node.execute(f"cat /home/ubuntu/drop_check_{name}.log")[0]
        
        # before_sender = parse_drop_log(before_sender_log)
        # after_sender  = parse_drop_log(after_sender_log)
        
        # # --- RECEIVER before/after ---
        # before_receiver_log = receiver_node.execute(f"cat /home/ubuntu/drop_check_{name}_before.log")[0]
        # after_receiver_log  = receiver_node.execute(f"cat /home/ubuntu/drop_check_{name}.log")[0]
        
        # before_receiver = parse_drop_log(before_receiver_log)
        # after_receiver  = parse_drop_log(after_receiver_log)
        
        # # --- Print comparisons ---
        # print_counter_diff("ROUTER",   before_router,   after_router)
        # print_counter_diff("SENDER",   before_sender,   after_sender)
        # print_counter_diff("RECEIVER", before_receiver, after_receiver)
        

        time.sleep(25) 
      
print("done")


# output = router_node.execute("ethtool -S enp8s0np1 | grep rx_out_of_buffer")[0].strip()
# print(f"router enp8s0np1: {output}")

# output = router_node.execute("ethtool -S enp7s0np0 | grep rx_out_of_buffer")[0].strip()
# print(f"router enp7s0np0: {output}")

# output = receiver_node.execute("ethtool -S enp8s0np1 | grep rx_out_of_buffer")[0].strip()
# print(f"receiver enp7s0np0: {output}")

# output = sender_node.execute("ethtool -S enp7s0 | grep rx_out_of_buffer")[0].strip()
# print(f"sender enp7s0np0: {output}")

In [ ]:
router_node.execute("sudo pkill -f tm10")
router_node.execute("pkill -f 'mpstat -P ALL 1'")

router_node.execute("sudo rm -f /mnt/huge/rte*map_*")

router_node.execute("sudo rm -f /mnt/huge1G/rte*map_*")

router_node.execute("sudo rm -f /dev/hugepages/rte*map_*")

In [ ]:
 # --- ROUTER before/after ---
before_router_log = router_node.execute(f"cat /home/ubuntu/drop_check_{name}_before.log")[0]
after_router_log  = router_node.execute(f"cat /home/ubuntu/drop_check_{name}.log")[0]

before_router = parse_drop_log(before_router_log)
after_router  = parse_drop_log(after_router_log)

# --- SENDER before/after ---
before_sender_log = sender_node.execute(f"cat /home/ubuntu/drop_check_{name}_before.log")[0]
after_sender_log  = sender_node.execute(f"cat /home/ubuntu/drop_check_{name}.log")[0]

before_sender = parse_drop_log(before_sender_log)
after_sender  = parse_drop_log(after_sender_log)

# --- RECEIVER before/after ---
before_receiver_log = receiver_node.execute(f"cat /home/ubuntu/drop_check_{name}_before.log")[0]
after_receiver_log  = receiver_node.execute(f"cat /home/ubuntu/drop_check_{name}.log")[0]

before_receiver = parse_drop_log(before_receiver_log)
after_receiver  = parse_drop_log(after_receiver_log)

# --- Print comparisons ---
print_counter_diff("ROUTER",   before_router,   after_router)
print_counter_diff("SENDER",   before_sender,   after_sender)
print_counter_diff("RECEIVER", before_receiver, after_receiver)

In [ ]:
cmds_py_install = '''
            sudo apt -y install python3-pip
            pip install numpy
            pip install matplotlib
            pip install pandas
            '''

sender_node.execute(cmds_py_install)
receiver_node.execute(cmds_py_install)

In [ ]:
data_dir="exp-results"

sender_node.execute('mkdir '+data_dir)
sender_node.execute('mv *.json '+ data_dir)

receiver_node.execute('mkdir '+data_dir)
receiver_node.execute('mv *.txt '+ data_dir)

In [ ]:
sender_node.upload_file("/home/fabric/work/Internship-work/analysis_simple.py", f"/home/ubuntu/{data_dir}/analysis.py")

sender_node.execute(f'python3 /home/ubuntu/{data_dir}/analysis.py')

outname1 = f"sender-throughput.json"

# remote source stays the same, local filename changes
sender_node.download_file(f"/home/fabric/work/Internship-work/L2P2P-QoS/{outname1}", f"/home/ubuntu/{data_dir}/throughput_data.json")

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# ----------------------------
# Config (paper-ready styling)
# ----------------------------
plt.rcParams.update({
    "font.size": 10,
    "axes.labelsize": 10,
    "axes.titlesize": 10,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
    "pdf.fonttype": 42,   # TrueType fonts in PDF
    "ps.fonttype": 42,
    "axes.linewidth": 0.8,
})

INPUT_JSON = "sender-throughput.json"
OUT_PDF = "sender_throughput_groups.pdf"
OUT_PNG = "sender_throughput_groups.png"  # optional raster copy

# Define suffix ranges and (ordered) labels
groups = [
    ("DPDK Shaping", range(250, 255)),
    ("DPDK Forwarding", range(350, 355)),
    ("Linux forwarding", range(450, 455)),
    ("HTB Shaping", range(550, 555)),
]

# ----------------------------
# Load + group data
# ----------------------------
with open(INPUT_JSON, "r") as f:
    data = json.load(f)

group_values = {name: [] for name, _ in groups}

for key, value in data.items():
    try:
        suffix = int(str(key).split("_")[-1])
    except (ValueError, AttributeError):
        continue

    for group_name, suffix_range in groups:
        if suffix in suffix_range:
            group_values[group_name].append(float(value) / 1000.0)  # Gb/s
            break

# Compute stats (skip empty groups)
labels = []
means = []
mins_ = []
maxs_ = []
ns = []

for group_name, _ in groups:
    vals = np.array(group_values[group_name], dtype=float)
    if vals.size == 0:
        continue
    labels.append(group_name)
    means.append(vals.mean())
    mins_.append(vals.min())
    maxs_.append(vals.max())
    ns.append(vals.size)

means = np.array(means)
mins_ = np.array(mins_)
maxs_ = np.array(maxs_)

# Asymmetric whiskers: [lower, upper]
yerr = np.vstack([means - mins_, maxs_ - means])

# ----------------------------
# Plot
# ----------------------------
fig, ax = plt.subplots(figsize=(5.5, 2.3), constrained_layout=True)

x = np.arange(len(labels))
bars = ax.bar(
    x, means,
    yerr=yerr, capsize=4,
    linewidth=0.6, edgecolor="black"
)

ax.set_ylabel("Average Throughput (Gb/s)")
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=0, ha="center")

# Subtle grid for readability
ax.yaxis.grid(True, linewidth=0.6, alpha=0.35)
ax.set_axisbelow(True)

# Annotate mean above each bar (and optionally N)
ymax = maxs_.max() if len(maxs_) else 1.0
for i, (m, lo, hi, n) in enumerate(zip(means, mins_, maxs_, ns)):
    ax.text(
        i,
        hi + 0.02 * ymax,          # place above max whisker
        f"{m:.2f}",                # or f"{m:.2f}\nN={n}"
        ha="center",
        va="bottom"
    )

# Tighten y-limits to include whiskers nicely
ax.set_ylim(0, ymax * 1.12)

# Save vector + optional PNG
Path(OUT_PDF).parent.mkdir(parents=True, exist_ok=True)
fig.savefig(OUT_PDF, bbox_inches="tight")
fig.savefig(OUT_PNG, dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
sender_node.download_file("/home/fabric/work/Internship-work/new-exps-paper/fct-cubic-bbr-1.6Gbps.json",f"/home/ubuntu/{data_dir}/fct_data.json")
sender_node.download_file("/home/fabric/work/Internship-work/new-exps-paper/retrans-cubic-bbr-1.6Gbps.json",f"/home/ubuntu/{data_dir}/retransmits_data.json")
sender_node.download_file("/home/fabric/work/Internship-work/new-exps-paper/rtt-cubic-bbr-1.6Gbps.json",f"/home/ubuntu/{data_dir}/stream_rtt_data.json")

In [ ]:
receiver_node.download_file("/home/fabric/work/Internship-work/cubic-bbr-1.6Gbps-receiver_stats.json",f"/home/ubuntu/{data_dir}/final_stream_stats.json")
receiver_node.download_file("/home/fabric/work/Internship-work/cubic-bbr-1.6Gbps-receiver_stats-jfi.json",f"/home/ubuntu/{data_dir}/fairness_index.json")

### Plot

In [ ]:
name="bbr_single-queue-FQ_1_150_8_100_52_10_100_5_0_0_3"

In [ ]:
sender_node.download_file(f"/home/fabric/work/Internship-work/L2P2P-QoS/{name}.json",f"/home/ubuntu/{name}.json")
receiver_node.download_file(f"/home/fabric/work/Internship-work/L2P2P-QoS/{name}.txt",f"/home/ubuntu/{name}.txt")

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# ----------------------------
# Paper-style plotting config
# ----------------------------
plt.rcParams.update({
    "font.size": 9,
    "axes.titlesize": 9,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "pdf.fonttype": 42,
    "ps.fonttype": 42,
    "axes.linewidth": 0.8,
})

TXT_PATH = f"{name}.txt"
JSON_PATH = f"{name}.json"
OUT_PDF = f"{name}_timeseries.pdf"

# ----------------------------
# Load goodput
# ----------------------------
times = []
rates = []

with open(TXT_PATH) as f:
    for line in f:
        if line.startswith("[SUM]") and "receiver" not in line:
            parts = line.split()
            start_time = float(parts[1].split("-")[0])
            rate = float(parts[-2])
            if parts[-1].startswith("Gbits"):
                rate *= 1000.0
            times.append(start_time)
            rates.append(rate / 1000.0)  # Gb/s

times = np.array(times)
rates = np.array(rates)

# ----------------------------
# Load JSON metrics
# ----------------------------
with open(JSON_PATH) as f:
    data = json.load(f)

metrics = defaultdict(lambda: defaultdict(list))

for interval in data["intervals"]:
    for s in interval["streams"]:
        sid = s["socket"]
        metrics[sid]["retrans"].append(s.get("retransmits", 0))
        metrics[sid]["rtt"].append(s.get("rtt", 0) / 1000)

T = max(len(v["retrans"]) for v in metrics.values())
t = np.arange(T)  # interval index (time proxy)

# ----------------------------
# Plot (3 panels)
# ----------------------------
fig, axes = plt.subplots(1, 3, figsize=(5.4, 2.0), constrained_layout=True)

def clean_axis(ax):
    ax.grid(True, linewidth=0.6, alpha=0.35)
    ax.set_axisbelow(True)

# (1) Goodput
axes[0].plot(times, rates, linewidth=1.0)
axes[0].set_title("Goodput (Gb/s)")
axes[0].set_xlabel("Time (seconds)")
clean_axis(axes[0])

# (2) Retransmissions
for sid in metrics:
    axes[1].plot(t[:len(metrics[sid]["retrans"])],
                 metrics[sid]["retrans"],
                 linewidth=1.0)
axes[1].set_title("Retransmissions")
axes[1].set_xlabel("Time (seconds)")
clean_axis(axes[1])

# (3) RTT
for sid in metrics:
    axes[2].plot(t[:len(metrics[sid]["rtt"])],
                 metrics[sid]["rtt"],
                 linewidth=1.0)
axes[2].set_title("RTT (ms)")
axes[2].set_xlabel("Time (seconds)")
clean_axis(axes[2])

# ----------------------------
# Save
# ----------------------------
fig.savefig(OUT_PDF, bbox_inches="tight")
plt.show()


### Delete your slice

When you finish your experiment, you should delete your slice! The following cells deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()